In [1]:
import os 

import pandas as pd
import numpy as np

"""
# import radial color histogram code, which can be found here:
# https://github.com/gmorinan/radialColorHistogram
from radialColorHistogram.color_density import ColorDensity
from radialColorHistogram.radial_split import RadialSplitter
"""

from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet as wn
from collections import defaultdict
import regex as re
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer

nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
nltk.download('stopwords')

try:
  import contractiond
except:
  !pip install contractions
  import contractions

tweet_tokenizer = TweetTokenizer()
# word_Lemmatized = WordNetLemmatizer()
stop = set(stopwords.words("english")) 
stemmer = SnowballStemmer(language="english")

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

try:
  from zeugma.embeddings import EmbeddingTransformer
except:
  !pip install zeugma
  from zeugma.embeddings import EmbeddingTransformer


from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

try:
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter
except:
  !pip install textattack
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter

from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 327kB 4.3MB/s 
     |████████████████████████████████| 286kB 7.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85391 sha256=818e8203df0a805589bfa286aecfe891832a2ef44131a1fa433cb5d928b14119
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
  Created wheel for zeugma: filename=zeugma-0.48-cp37-none-any.whl size=8778 sha256=9549a552c26d49bf3f341a9da4854b53e3e619481d4238aed59655126057fa7f
  Stored in directory: /root/.cache/pip/wheels/6a/b5/bc/5183ac478b0071d04d3ed0c0dd4a43db94c5c8ffb317b5eb53
Successfully built zeugma
     |████████████████████████████████| 358kB 5.7MB/s 
     |███████████████

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 44.7MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpnq747grf.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


### PP

In [2]:
root = "/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/"
train_path = "train_images/train_images"
test_path = "test_images"

train_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train.csv")
test_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/test.csv")
train_dt.head()

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0
3,4,448_image_batch_2.png,"""COME ON MAN, YOU KNOW THE THING.\r\nJUST ASK ...",troll,2
4,5,653_image_batch_2.png,"""Those who believe without reason cannot be co...",none,0


In [58]:
def preprocess_txt(text):

  caps = True if text != text.lower() else False
  multiline = True if '\n' in text else False
  bullet = True if '- ' in text else False

  text = text.lower()   # ??
  text = re.sub(r"\n"," ",text)
  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)
  
  # contractions fis
  expanded_words = []
  for word in text.split():
    expanded_words.append(contractions.fix(word))
  text = " ".join(expanded_words)

  """
  #$# text = text.replace(r'\s+', ' ')
  text = text.replace("@","@ ")
  text = re.sub(r'[^a-zA-Z0-9 \@]', ' ', text) # ????
  ## text = re.sub('@[^\s]+','',text)
  """

  text = re.sub(r'[^a-zA-Z0-9 \.\,\-]', ' ', text) # ????

  # Single character removal
  text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

  # Removing multiple spaces
  text = re.sub(r'\s+', ' ', text)

  #####text = tweet_tokenizer.tokenize(text)
  text = word_tokenize(text)
  text = [x for x in text if x not in stop]
  
  text = [stemmer.stem(x) for x in text]

  ##text = ["<CAPS>"] + text if caps else text
  text = ["<MULTILINE>"] + text if multiline else text
  ##text = ["<BULLET>"] + text if bullet else text


  text = " ".join(text)


  return text


def preprocess_txt_mini(text):

  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)

  text = re.sub(r'[^a-zA-Z0-9 \.,\?\\\'\"\:\;\#\%\!\&\-\@\n]', ' ', text) # ????

  return text


def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(stopwords.words("english")) 

In [59]:
def get_files_labels(subpath,csv):
    """Gets paths images within a folder, and their labels"""
    img_paths, img_labels, texts = [], [], []
    df = pd.read_csv(root+csv)
    i=0
    for index, row in df.iterrows():
        try:
          img, text, label = row["image id"], row["text"], int(row["label_num"])
        except:
          img, text, label = row["image id"], row["text"], -1
        # print(img_,": ",text_)
        
        ## Change this for img....>>>
        ##$$# if not (i in [858, 1986, 1990] and csv=="train.csv"):
        if True:
          img_paths.append(root+subpath+'/'+img.strip())
          img_labels.append(label)
          ###texts.append(preprocess_txt(text))
          ###texts.append(text)
          texts.append(preprocess_txt_mini(text))
        i+=1
        #print('\r images: {} labels : {}'.format(len(img_paths), len(img_labels)) )
    return img_paths, img_labels, texts

In [60]:
files_, y_, texts_ = get_files_labels(train_path,"train.csv")
files_test, y_test, texts_test = get_files_labels(test_path,"test.csv")


In [61]:
X_train, X_val, y_train, y_val = train_test_split(
    np.column_stack([files_, texts_]), y_, test_size=0.1, random_state=42, stratify=y_, shuffle=True)
X_test = np.column_stack([files_test,texts_test])
# X_train[0]

In [62]:
##aug = EasyDataAugmenter()
##aug.augment(texts_[0])

In [63]:
#X_val1 = X_val.copy()
#np.all(X_val == X_val1)

In [64]:
def newaug(sentences,aug = EmbeddingAugmenter(),easy=False ):
  new_sentences = []
  for sentence in sentences:
    if easy:
      sen = aug.augment(sentence)
      idx,le = 0,0
      for i in range(len(sen)):
        if len(sen[i]) > le:
          le = len(sen[i])
          idx = i
      new_sentences.append(sen[idx])     
    else:
      new_sentences.append(aug.augment(sentence)[0])
  return new_sentences

# newaug(X_train[:,1])[0]
# newaug(["HELLO from New York!","I'm fine!"],WordNetAugmenter())

In [65]:
X_train_wn = newaug(X_train[:,1], aug=WordNetAugmenter())
print("Done0")
#X_train_emb = newaug(X_train[:,1], aug=EmbeddingAugmenter())
#print("Done1")
#X_train_ed = newaug(X_train[:,1], aug=EasyDataAugmenter(), easy=True)


Done0


In [66]:
"""
import pickle 

open_file = open("04-25-modpp__X_train_wn.pkl", "rb")
X_train_wn = pickle.load(open_file)
open_file.close()
"""

'\nimport pickle \n\nopen_file = open("04-25-modpp__X_train_wn.pkl", "rb")\nX_train_wn = pickle.load(open_file)\nopen_file.close()\n'

In [67]:
## print(X_train[0,1])
for i in range(X_train.shape[0]):
  X_train[i,1] = preprocess_txt(X_train[i,1])

for i in range(X_val.shape[0]):
  X_val[i,1] = preprocess_txt(X_val[i,1])

for i in range(X_test.shape[0]):
  X_test[i,1] = preprocess_txt(X_test[i,1])

In [68]:
X_train_wn[0]

'20 years later and all of these things burst in your pocket. nfigueiroa   9GAG HOLI L500 LINI NORACY IX Follow @9GAG on Instagram!'

In [69]:
for i in range(X_train.shape[0]):
  X_train_wn[i] = preprocess_txt(X_train_wn[i])
  #X_train_emb[i] = preprocess_txt(X_train_emb[i])
  #X_train_ed[i] = preprocess_txt(X_train_ed[i])


In [70]:
import pickle 
"""
open_file = open("X_train_emb1.pkl", "rb")
X_train_emb = pickle.load(open_file)
open_file.close()


open_file = open("X_train_wn.pkl", "rb")
X_train_wn = pickle.load(open_file)
open_file.close()
"""

'\nopen_file = open("X_train_emb1.pkl", "rb")\nX_train_emb = pickle.load(open_file)\nopen_file.close()\n\n\nopen_file = open("X_train_wn.pkl", "rb")\nX_train_wn = pickle.load(open_file)\nopen_file.close()\n'

In [71]:
X_train_wn[0]

'20 year later thing burst pocket . nfigueiroa 9gag holi l500 lini noraci ix follow 9gag instagram'

In [72]:
def augment(sentences,n):
    import random
    random.seed(42)
    new_sentences = []
    for _ in range(n):
      for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        new_sentences.append(' '.join(words))
    return new_sentences

In [73]:
def get_synonyms_lexicon(path="ppdb-xl.txt"):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path).readlines()]
    for e in text_entries:
        e = e.split(' ')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

synonyms = get_synonyms_lexicon()

In [74]:
X_train2 = augment(X_train[:,1],1)
X_train[:,1][0],X_train2[0]

('20 year later thing fit pocket . nfigueiroa 9gag holi l500 lini noraci 9 follow 9gag instagram',
 'nfigueiroa l500 pocket . 9gag follow 9 holi 9gag year later noraci instagram fit lini 20 thing')

In [75]:
X_train1 = []
for sentence in X_train[:,1]:
  new_sentence = []
  for word in sentence.split():
    if word in synonyms.keys():
      new_sentence.append(synonyms[word][0])
    else:
      new_sentence.append(word)
  X_train1.append(" ".join(new_sentence))

X_train1[0]

'20 annum later thing appropriate pocket ... nfigueiroa 9gag holi l500 lini noraci nine follow 9gag instagram'

In [21]:
def img_col(X):
    return X[:,0]

def text_col(X):
    return X[:,1]


In [32]:
preprocess_txt("ali imgflip.com")

'ali'

In [33]:
tweet_tokenizer.tokenize(text)

NameError: ignored

In [ ]:
X_train_______ = X_train.copy()
X_train1_______ = X_train1.copy()

In [ ]:
for i in range(X_train.shape[0]):
  if X_train[i,1] != X_train_______[i,1]:
    print(i, '\n', X_train[i,1],'\n', X_train_______[i,1])
    break


In [ ]:
for i in range(X_train.shape[0]):
  if  X_train1[i] != X_train1_______[i]:
    print(i, '\n', X_train1[i],'\n', X_train1_______[i])
    break


### Text

In [201]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import shuffle

"""
text_pipe = Pipeline([
                     ("tdidf",TfidfVectorizer(
                        min_df=4, norm='l2', 
                        encoding='utf-8', 
                        ngram_range=(1,   ))),
                     ("svc",SVC(kernel='linear',probability=True, C=10, max_iter=50000, random_state=2020))
                    ],
                    verbose = 1)
"""

"""
text_pipe = Pipeline([
        #('vect', CountVectorizer()),
        #('tfidf', TfidfTransformer()),
        ("tdidf",TfidfVectorizer(
                        min_df=3, norm='l2', 
                        encoding='utf-8', 
                        lowercase = True,
                        ngram_range=(1, 2),
                        preprocessor = None)),
        ('clf', SVC(C=2,random_state=42,probability=True))
        ],
        verbose = 1)
"""
######################################


## BEST
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = None,
                  min_df=3, max_df=600,
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', SVC(C=1, max_iter=5000, random_state=42, probability=True)) ],
        verbose = 1)


"""
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = None,
                  min_df=5, max_df=600,
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', MultinomialNB() ) ],
        verbose = 1)
"""


'''
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 3),
                  preprocessor = None,
                  min_df=9, 
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', XGBClassifier(
    gamma=4,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
) )],
        verbose = 1)
'''

"""
### emb = EmbeddingTransformer('fasttext')

text_pipe = Pipeline([
        # ('pp', FunctionTransformer(preprocess_txt) ),
        ('vect', EmbeddingTransformer('glove') ),  # glove, word2vec, fasttext
        ('clf', SVC(C=2, random_state=42) )],
        verbose = 1)
"""

'''
text_pipe = Pipeline([
        # ('pp', FunctionTransformer(preprocess_txt) ),
        ('vect', EmbeddingTransformer('glove') ),  # glove, word2vec, fasttext
        # ('clf', LogisticRegression(class_weight="balanced", random_state=42),
        ('clf', OneVsRestClassifier( LogisticRegression(solver="sag", class_weight="balanced", random_state=42) ) )
        ],

        verbose = 1)
'''

'''
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = None,
                  min_df=3, max_df=600,
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        # ('clf', LogisticRegression(class_weight="balanced", random_state=42),
        ('clf', OneVsRestClassifier( LogisticRegression(class_weight="balanced", random_state=42) ) 
        )],
        verbose = 1)

'''

_XX_, _YY_ = shuffle( list( X_train[:,1])+X_train1+ X_train_wn ,3*[x for x in y_train] , random_state=42)
text_pipe.fit( _XX_,_YY_ )

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  25.8s


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=600,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
 

In [202]:
# make predictions
y_pred_train_text = text_pipe.predict(X_train[:,1])
y_pred_val_text = text_pipe.predict(X_val[:,1])
y_pred_test_text = text_pipe.predict(X_test[:,1])

In [203]:
from sklearn.metrics import classification_report
print(classification_report([x for x in y_train ], y_pred_train_text))
print(classification_report([x for x in y_val], y_pred_val_text))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           0.99      1791
   macro avg       0.99      0.99      0.99      1791
weighted avg       0.99      0.99      0.99      1791

              precision    recall  f1-score   support

           0       0.38      0.26      0.31        61
           1       0.42      0.52      0.46        69
           2       0.42      0.43      0.42        70

    accuracy                           0.41       200
   macro avg       0.41      0.40      0.40       200
weighted avg       0.41      0.41      0.40       200



In [ ]:
print(*y_pred_test_text,sep="\n")

In [80]:
ftxt_v = text_pipe.predict_proba(X_val[:,1])
ftxt_t = text_pipe.predict_proba(X_test[:,1])

AttributeError: ignored

In [ ]:
import pickle

open_file = open("lstm__val_pred_2.pkl", "rb")
flstm_v = pickle.load(open_file)
open_file.close()

open_file = open("lstm__test_pred_2.pkl", "rb")
flstm_t = pickle.load(open_file)
open_file.close()

In [ ]:
f = np.argmax(ftxt_v+fimg_v+flstm_v ,axis=1)
f.shape

(199,)

In [ ]:
print(classification_report([x for x in y_val], f))

In [ ]:
ft = np.argmax(ftxt_t+fimg_t+flstm_t ,axis=1)
print(*ft,sep='\n')

In [ ]:
[x.replace("@ ","") for x in list( X_train[:,1])+X_train_wn+X_train1] ==  list( X_train[:,1])+X_train_wn+X_train1

False

#### Pipeline Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV


tuned_parameters = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3, 1e-4],
                     'svc__C': [1, 10, 100, 1000]},
                    {'svc__kernel': ['linear'], 'svc__C': [1, 10, 100, 1000]}]


text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = preprocess_txt,) ),
        ('tfidf', TfidfTransformer()),
        ('svc', SVC(random_state=42)) ],
        verbose = 1)



clf = GridSearchCV(
    text_pipe, tuned_parameters, scoring='f1_macro',
    refit=True,verbose=3
)

clf.fit(X_train[:,1],y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.174, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.3s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173,

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.2min finished


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.6s
Best parameters set found on development set:

{'svc__C': 1000, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}

Grid scores on development set:

0.173 (+/-0.001) for {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 10, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.198 (+/-0.017) for {'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 100, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.395 (+/-0.049) for {'svc__C': 1000, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.198 (+/-0.017) for {'svc__C': 1000, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.392 

In [ ]:
# make predictions
y_pred_train_text = clf.predict(X_train[:,1])
y_pred_val_text = clf.predict(X_val[:,1])
y_pred_test_text = clf.predict(X_test[:,1])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train_text))
print(classification_report(y_val, y_pred_val_text))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       513
           1       1.00      1.00      1.00       586
           2       1.00      0.99      1.00       593

    accuracy                           1.00      1692
   macro avg       1.00      1.00      1.00      1692
weighted avg       1.00      1.00      1.00      1692

              precision    recall  f1-score   support

           0       0.30      0.26      0.28        91
           1       0.43      0.49      0.46       103
           2       0.39      0.39      0.39       105

    accuracy                           0.38       299
   macro avg       0.38      0.38      0.38       299
weighted avg       0.38      0.38      0.38       299



In [ ]:
## text_pipe.predict_proba(X_val[:,1])

In [ ]:
y_pred_test

In [ ]:
print(*y_pred_test_text,sep="\n")

### Combine

In [ ]:
y_pred_train_comb = np.column_stack([y_pred_train_img,y_pred_train_text])
y_pred_val_comb = np.column_stack([y_pred_val_img,y_pred_val_text])
#y_pred_test_comb = np.column_stack([y_pred_test_img,y_pred_test_text])


In [ ]:
#clf = DecisionTreeClassifier()
#clf = RandomForestClassifier()
clf = SVC(random_state=42)
clf = clf.fit(y_pred_train_comb, y_train)


In [ ]:
# make predictions
y_pred_train_comb__ = clf.predict(y_pred_train_comb)
y_pred_val_comb__ = clf.predict(y_pred_val_comb)
#y_pred_test_comb__ = clf.predict(y_pred_test_comb)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train_comb__))
print(classification_report(y_val, y_pred_val_comb__))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       543
           1       0.88      0.89      0.89       618
           2       0.86      0.89      0.87       628

    accuracy                           0.88      1789
   macro avg       0.88      0.88      0.88      1789
weighted avg       0.88      0.88      0.88      1789

              precision    recall  f1-score   support

           0       0.38      0.33      0.36        60
           1       0.48      0.52      0.50        69
           2       0.40      0.41      0.41        70

    accuracy                           0.43       199
   macro avg       0.42      0.42      0.42       199
weighted avg       0.42      0.43      0.42       199



In [ ]:
print(*y_pred_test_comb__,sep="\n")

## Misc

In [84]:
import pickle

open_file = open("__latest__4-25__X_train_wn1.pkl", "wb")
pickle.dump(X_train_wn, open_file)
open_file.close()





In [204]:


open_file = open("__latest__4-25__text_pipe1_prob.sav", "wb")
pickle.dump(text_pipe, open_file)
open_file.close()



In [ ]:
iii=3
X_train_wn[iii], X_train_emb[iii]

('mather cri franchis domino pizza make offer refus 03 16 parmesan lettuc burn 1 2 medium 2 top pizza 5 99 godfath trilog godfath crono 2 p ort rount corpor alagam',
 'mather call franchis domino pizza make offer refus 03 16 parmesan bread bite 1 2-medium 2-top pizza onto 5 99 godfath triad godfath crono 2 p ort rount firm alagam')

In [82]:


open_file = open("__cnn_y_val.pkl", "rb")
y__val_ = pickle.load( open_file)
open_file.close()



In [83]:
y__val_ == y_val

True

In [132]:
open_file = open("__latest__4-25__X_train.pkl", "wb")
pickle.dump(X_train, open_file)
open_file.close()

open_file = open("__latest__4-25__X_test.pkl", "wb")
pickle.dump(X_test, open_file)
open_file.close()

open_file = open("__latest__4-25__X_val.pkl", "wb")
pickle.dump(X_val, open_file)
open_file.close()

open_file = open("__latest__4-25__y_train.pkl", "wb")
pickle.dump(y_train, open_file)
open_file.close()

open_file = open("__latest__4-25__y_val.pkl", "wb")
pickle.dump(y_val, open_file)
open_file.close()

# Ensemble

## My

In [150]:
open_file = open("__latest__4-25__text_pipe.sav", "rb")
clf = pickle.load( open_file)
open_file.close()


# make predictions
y_pred_train_text1 = clf.predict(list(X_train[:,1]))
y_pred_val_text1 = clf.predict(X_val[:,1])

print(classification_report([x for x in y_train], y_pred_train_text1))
print(classification_report([x for x in y_val], y_pred_val_text1))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       543
           1       0.99      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           0.99      1791
   macro avg       0.99      0.99      0.99      1791
weighted avg       0.99      0.99      0.99      1791

              precision    recall  f1-score   support

           0       0.34      0.26      0.30        61
           1       0.43      0.51      0.47        69
           2       0.42      0.43      0.42        70

    accuracy                           0.41       200
   macro avg       0.40      0.40      0.40       200
weighted avg       0.40      0.41      0.40       200



In [196]:
clf

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=600,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
 

In [353]:
open_file = open("__latest__4-25__text_pipe1_prob.sav", "rb")
clf = pickle.load( open_file)
open_file.close()



# make predictions
y_pred_val_text2 = clf.predict(X_val[:,1])
y_pred_train_text2 = clf.predict(list(X_train[:,1]))

y_pred_val_text2_ = clf.predict_proba(X_val[:,1])
y_pred_train_text2_ = clf.predict_proba(list(X_train[:,1]))

y_pred_test_text2_ = clf.predict_proba(list(X_test[:,1]))


print(classification_report([x for x in y_train], y_pred_train_text2))
print(classification_report([x for x in y_val], y_pred_val_text2))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           0.99      1791
   macro avg       0.99      0.99      0.99      1791
weighted avg       0.99      0.99      0.99      1791

              precision    recall  f1-score   support

           0       0.38      0.26      0.31        61
           1       0.42      0.52      0.46        69
           2       0.42      0.43      0.42        70

    accuracy                           0.41       200
   macro avg       0.41      0.40      0.40       200
weighted avg       0.41      0.41      0.40       200



In [137]:
with open("out/my_1.txt","w") as f:
  print(*y_pred_val_text1,sep="\n",file=f)

In [138]:
with open("out/my_2.txt","w") as f:
  print(*y_pred_val_text2,sep="\n",file=f)

## Transfer

In [141]:
open_file = open("__cnn_pred_val.pkl", "rb")
__cnn_pred_val = pickle.load( open_file)
open_file.close()

open_file = open("__cnn_y_val.pkl", "rb")
__cnn_y_val = pickle.load( open_file)
open_file.close()


open_file = open("__cnn_pred_train.pkl", "rb")
__cnn_pred_train = pickle.load( open_file)
open_file.close()

open_file = open("__cnn_y_train.pkl", "rb")
__cnn_y_train = pickle.load( open_file)
open_file.close()

In [101]:
__cnn_y_val == y_val

True

In [355]:
open_file = open("nlptone__dt.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t0 = clf.predict(__cnn_pred_val)
y_pred_train_text_t0 = clf.predict(__cnn_pred_train)

y_pred_train_test_t0 = clf.predict(__cnn_pred_test)


print(classification_report([x for x in y_train], y_pred_train_text_t0))
print(classification_report([x for x in y_val], y_pred_val_text_t0))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.35      0.36      0.36        61
           1       0.42      0.43      0.43        69
           2       0.45      0.43      0.44        70

    accuracy                           0.41       200
   macro avg       0.41      0.41      0.41       200
weighted avg       0.41      0.41      0.41       200



In [356]:
open_file = open("nlptone__dt1.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t1 = clf.predict(__cnn_pred_val)
y_pred_train_text_t1 = clf.predict(__cnn_pred_train)

y_pred_train_test_t1 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t1))
print(classification_report([x for x in y_val], y_pred_val_text_t1))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.37      0.36      0.37        61
           1       0.40      0.41      0.40        69
           2       0.46      0.47      0.47        70

    accuracy                           0.41       200
   macro avg       0.41      0.41      0.41       200
weighted avg       0.41      0.41      0.41       200



In [357]:
open_file = open("nlptone__dt2.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t2 = clf.predict(__cnn_pred_val)
y_pred_train_text_t2 = clf.predict(__cnn_pred_train)

y_pred_train_test_t2 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t2))
print(classification_report([x for x in y_val], y_pred_val_text_t2))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.36      0.39      0.38        61
           1       0.41      0.43      0.42        69
           2       0.46      0.40      0.43        70

    accuracy                           0.41       200
   macro avg       0.41      0.41      0.41       200
weighted avg       0.41      0.41      0.41       200



In [358]:
open_file = open("nlptone__dt3.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t3 = clf.predict(__cnn_pred_val)
y_pred_train_text_t3 = clf.predict(__cnn_pred_train)

y_pred_train_test_t3 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t3))
print(classification_report([x for x in y_val], y_pred_val_text_t3))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.36      0.33      0.34        61
           1       0.41      0.39      0.40        69
           2       0.46      0.51      0.49        70

    accuracy                           0.41       200
   macro avg       0.41      0.41      0.41       200
weighted avg       0.41      0.41      0.41       200



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [359]:
open_file = open("nlptone__dt4.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t4 = clf.predict(__cnn_pred_val)
y_pred_train_text_t4 = clf.predict(__cnn_pred_train)

y_pred_train_test_t4 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t4))
print(classification_report([x for x in y_val], y_pred_val_text_t4))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.38      0.34      0.36        61
           1       0.41      0.43      0.42        69
           2       0.43      0.43      0.43        70

    accuracy                           0.41       200
   macro avg       0.40      0.40      0.40       200
weighted avg       0.40      0.41      0.40       200



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [360]:
open_file = open("nlptone__dt5.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t5 = clf.predict(__cnn_pred_val)
y_pred_train_text_t5 = clf.predict(__cnn_pred_train)

y_pred_train_test_t5 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t5))
print(classification_report([x for x in y_val], y_pred_val_text_t5))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.35      0.36      0.35        61
           1       0.42      0.41      0.41        69
           2       0.49      0.49      0.49        70

    accuracy                           0.42       200
   macro avg       0.42      0.42      0.42       200
weighted avg       0.42      0.42      0.42       200



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [361]:
open_file = open("nlptone__dt6.sav", "rb")
clf = pickle.load( open_file)
open_file.close()

# make predictions
y_pred_val_text_t6 = clf.predict(__cnn_pred_val)
y_pred_train_text_t6 = clf.predict(__cnn_pred_train)

y_pred_train_test_t6 = clf.predict(__cnn_pred_test)

print(classification_report([x for x in y_train], y_pred_train_text_t6))
print(classification_report([x for x in y_val], y_pred_val_text_t6))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      0.99       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.38      0.41      0.40        61
           1       0.40      0.42      0.41        69
           2       0.46      0.41      0.44        70

    accuracy                           0.41       200
   macro avg       0.42      0.41      0.41       200
weighted avg       0.42      0.41      0.42       200



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [140]:
with open("out/transfer_0.txt","w") as f:
  print(*y_pred_val_text_t0,sep="\n",file=f)

with open("out/transfer_1.txt","w") as f:
  print(*y_pred_val_text_t1,sep="\n",file=f)

with open("out/transfer_2.txt","w") as f:
  print(*y_pred_val_text_t2,sep="\n",file=f)

with open("out/transfer_3.txt","w") as f:
  print(*y_pred_val_text_t3,sep="\n",file=f)

with open("out/transfer_4.txt","w") as f:
  print(*y_pred_val_text_t4,sep="\n",file=f)

with open("out/transfer_5.txt","w") as f:
  print(*y_pred_val_text_t5,sep="\n",file=f)

with open("out/transfer_6.txt","w") as f:
  print(*y_pred_val_text_t6,sep="\n",file=f)

## Mixin

In [417]:
"""

new_X_train = np.column_stack([y_pred_train_text2_,
                               __cnn_pred_train
                               ])

new_X_val = np.column_stack([y_pred_val_text2_,
                             __cnn_pred_val
                             ])
"""

new_X_train = np.column_stack([y_pred_train_text2_,
                               y_pred_train_text_t6,
                               y_pred_train_text_t3,
                               y_pred_train_text_t2
                               ])

new_X_val = np.column_stack([y_pred_val_text2_,
                             y_pred_val_text_t6,
                             y_pred_val_text_t3,
                             y_pred_val_text_t2
                             ])

new_X_test = np.column_stack([y_pred_test_text2_,
                             y_pred_train_test_t6,
                             y_pred_train_test_t3,
                             y_pred_train_test_t2
                             ])

In [418]:
new_X_train.shape, new_X_test.shape

((1791, 6), (600, 6))

In [440]:
ensemble  = DecisionTreeClassifier()
ensemble.fit(new_X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [441]:
# make predictions
new_y_pred_train = ensemble.predict(new_X_train)
new_y_pred_val = ensemble.predict(new_X_val)

print(classification_report([x for x in y_train], new_y_pred_train))
print(classification_report([x for x in y_val], new_y_pred_val))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       543
           1       1.00      1.00      1.00       620
           2       1.00      0.99      1.00       628

    accuracy                           1.00      1791
   macro avg       1.00      1.00      1.00      1791
weighted avg       1.00      1.00      1.00      1791

              precision    recall  f1-score   support

           0       0.38      0.39      0.38        61
           1       0.41      0.46      0.44        69
           2       0.47      0.39      0.42        70

    accuracy                           0.41       200
   macro avg       0.42      0.41      0.41       200
weighted avg       0.42      0.41      0.41       200



In [340]:
while True:
  pass

KeyboardInterrupt: ignored

In [212]:
with open("out/y_val.txt","w") as f:
  print(*y_val,sep="\n",file=f)

with open("out/x_val.txt","w") as f:
  print(*X_val,sep="\n",file=f)



In [351]:
open_file = open("__cnn_pred_test.pkl", "rb")
__cnn_pred_test = pickle.load( open_file)
open_file.close()

In [398]:
__cnn_pred_test.shape

(600, 5)

In [399]:
y_pred_test_text2_.shape

(600, 3)

In [367]:
y_pred_train_test_t1.shape

(600,)

In [442]:
new_X_test.shape

(600, 6)

In [443]:
file_name_end = "pred/632__384442_41_2"

p_t = ensemble.predict(new_X_test)


with open(file_name_end+".txt","w") as f:
  print(*p_t,sep="\n",file=f)


open_file = open(file_name_end+".sav", "wb")
pickle.dump(ensemble, open_file)
open_file.close()